# Initial Python Setup

In [ ]:
import os
import pandas as pd
import requests as req
import numpy as np
import statsmodels.api as stapi
import statsmodels.formula.api as smf

from zipfile import ZipFile
from io import BytesIO
from IPython.display import display
from matplotlib import rc as pltrc
from matplotlib import pyplot as plt
from statsmodels.formula.api import ols
from statsmodels.regression.linear_model import RegressionResults, RegressionResultsWrapper

# Initial Variables
Here, we just set up initial variables and configuration to clean up code and results

In [ ]:
url = 'https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2022.zip'
file_name = 'survey_results_public.csv'

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 75)

font = {
    'family': 'normal',
    'weight': 'bold',
    'size': 10
}
pltrc('font', size=8)

# Importing the Dataset
Here we download and import the dataset in memory to eliminate the need of local storage.

In [ ]:
response = req.get(url)
zip_file = ZipFile(BytesIO(response.content))
df = pd.read_csv(BytesIO(zip_file.read(file_name)))
display(df)

# Cleaning the Data

In order to use the dataset, we will first clean it up and interpret some of the variables into dummy variables.

## Dummy Variables

Many of the variables in the data are multi-selected lists, where the user can choose none, any, or all of the options. These are stored as a list seperated by a `;` deliminer.

We can use the dataframe method

```python
df.str.get_dummies(';')
```

to split these up into dummy variables. We will keep the individual data frames so we can access all of the columns with

```python
list(dummy_df.columns)
```

Additionally, we will rename some columns in this step in order to make it usable in regressions

In [ ]:
dummy_languages = df["LanguageHaveWorkedWith"].str.get_dummies(';').rename(columns={
  "C#": "CSharp",
  "F#": "FSharp",
  "Bash/Shell": "Bash",
  "HTML/CSS": "HTML",
  "Objective-C": "ObjectiveC"
})
dummy_devtype = df["DevType"].str.get_dummies(";").rename(columns={
  "Academic researcher": "AcademicResearcher",
  "Cloud infrastructure engineer": "CloudInfrastructureEngineer",
  "Data or business analyst": "DataBusinessAnalyst",
  "Data scientist or machine learning specialist": "DataScientistMachineLearning",
  "Database administrator": "DatabaseAdministrator",
  "DevOps specialist": "DevOpsSpecialist",
  "Developer, QA or test": "DevQATest",
  "Developer, back-end": "DevBackend",
  "Developer, desktop or enterprise applications": "DevDesktopApps",
  "Developer, embedded applications or devices": "DevEmbedded",
  "Developer, front-end": "DevFrontend",
  "Developer, full-stack": "DevFullstack",
  "Developer, game or graphics": "DevGame",
  "Developer, mobile": "DevMobile",
  "Engineer, data": "DataEngineer",
  "Engineer, site reliability": "SiteReliabilityEngineer",
  "Engineering manager": "EngineerManager",
  "Marketing or sales professional": "MarketingSales",
  "Other (please specify):": "Other",
  "Product manager": "ProductManager",
  "Project manager":"ProjectManager",
  "Security professional": "Security",
  "Senior Executive (C-Suite, VP, etc.)": "SeniorExec",
  "System administrator": "SystemAdmin"
}).add_prefix("DevType")
dummy_employment = df["Employment"].str.get_dummies(";").rename(columns={
  "Employed, full-time": "FullTime",
  "Employed, part-time": "PartTime",
  "I prefer not to say": "Unknown",
  "Independent contractor, freelancer, or self-employed": "SelfEmployed",
  "Not employed, and not looking for work": "UnemployedNotLooking",
  "Not employed, but looking for work": "UnemployedLooking",
  "Student, full-time": "StudentFullTime",
  "Student, part-time": "StudentPartTime"
}).add_prefix("Employed")

df = pd.concat([df,dummy_languages,dummy_devtype,dummy_employment], axis=1)
display(df)

## Converting "Other" entries to their numbers
Some of the columns that we use are numbers, but they also include values such as "less than 1 year", etc. For that reason, we'll be replacing those values with values we can use to regress easier

In [ ]:
df["YearsCodeNumeric"] = pd.to_numeric(df["YearsCode"].replace({
  "Less than 1 year": 0,
  "More than 50 years": 50
}))
df["YearsCodeProNumeric"] = pd.to_numeric(df["YearsCodePro"].replace({
  "Less than 1 year": 0,
  "More than 50 years": 50
}))

## Filtering Data

Because the data is so vast across multiple countries and currencies, we will need to filter the data down to only rows that has everything we want in the format that we want, we'll store this filtered data separately as `dfreg`

In [ ]:
dfreg = df[
  (df["Country"] == "United States of America") &
  (df["ConvertedCompYearly"].notna())
]
display(dfreg)

# Running Regressions
Below will be regressions walking through the process to get to the final regression used for analysis

First, we'll make a string variable for each of the dummy sets that just prints each of the columns in that segment, joined by addition

In [ ]:
var_languages = " + ".join(list(dummy_languages.columns))
var_devtype = " + ".join(list(dummy_devtype.columns))
var_employment = " + ".join(list(dummy_employment.columns))

# Helper Functions


## Visualizing Results
This function will help with visualizing the results that we see in each outcome, specifically the relationship between language and salary

In [ ]:
def visualize_regression(regression: RegressionResultsWrapper):
  table = regression.summary2().tables[1]
  table = table[table.index.isin(list(dummy_languages.columns))]
  table = table.sort_values(by=["Coef."])
  plt.figure(figsize=(25,5))
  plt.xticks(rotation=90)
  plt.bar(table.index,table["Coef."],width=25/len(list(dummy_languages.columns)))
  plt.ylabel("Coefficient")
  plt.xlabel("Language (From lowest to highest salary impact)")

## The Basic Regression
The most basic regression that we can do is just modeling how the languages affects yearly compensation

In [ ]:
ols_basic = ols(f"ConvertedCompYearly ~ {var_languages}", data = dfreg).fit()
visualize_regression(ols_basic)

## Logging Converted Comp Yearly
By logging converted comp yearly, our interpretation of the coefficients is that knowing that particular language will result in a X% increase in the compensation

In [ ]:
ols_logsalary = ols(f"np.log(ConvertedCompYearly) ~ {var_languages}", data = dfreg).fit()
visualize_regression(ols_logsalary)

# Controlling for Developer Type

In [ ]:
ols_devtype = ols(f"np.log(ConvertedCompYearly) ~ {var_languages} + {var_devtype}", data = dfreg).fit()
visualize_regression(ols_devtype)

## Controlling for just employment status

In [ ]:
ols_employed = ols(f"np.log(ConvertedCompYearly) ~ {var_languages} + {var_employment}", data = dfreg).fit()
visualize_regression(ols_employed)

## CONTROLLING FOR EVERYTHING

In [ ]:
ols_all = ols(f"np.log(ConvertedCompYearly) ~ {var_languages} + {var_employment} + {var_employment} + YearsCodeNumeric +	YearsCodeProNumeric + EdLevel", data = dfreg).fit()
display(ols_all.summary())
visualize_regression(ols_all)